# Notebook de Generación de Versos
En este fichero enfocado para la entrega final del proyecto, experimentaremos con diferentes modelos de generación de texto. Donde nuestro objetivo va a ser evaluar diferentes modelos para ver cuales podrian aportar mejor contenido y explicaciones en relación al Corán.

El enfoque que tomaremos será realizar comparaciones entre modelos generativos especializados en el árabe y modelos generativos generalistas (que sirvan para muchos los idiomas).

Por otra parte, usaremos una **Large Language Model** (`LLM`) para evaluar la cohesión, coherencia y otras demás métricas de los versos generados. Dado el caso de que no haya ningún modelo evaluador árabe disponible, implementaremos otras *LLMs* manualmente para realizar esta segunda tarea.

## Dependencias

Aqui cargamos el HF_TOKEN que basicamente es la clave de acceso a la API de Hugging Face para poder usar los modelos. Además, usamos AutoTokenizer y AutoModelForCausalLM que nos dará el modelo y su tokenizer correspondiente al pasarle el nombre en from_pretained.

In [1]:
# Dependencias
import torch
from decouple import config
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
login(config("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


## Generación de Aleyas

Como ya hemos mencionado previamente, usaremos este apartado para evaluar si los modelos generalistas son capaces de hacer mejor language modeling que los modelos especializados en arabe.

Como nota diremos que aparte de las pruebas que se hacen aqui, en la sección de generación de topics + clustering ya se comprueba que modelos multilenguaje como ollama-gemma3:4b generan bastante bien texto en arabe. No solo eso sino que sus traducciones parecen ser bastante fieles despues de haberlas traducido manualmente. Sin embargo, aunque las traducciones fueran buenas, las comparaciones de cosine-similarity eran bastante inconcluyentes. Esto es debido a que probablemente la forma en la que creamos los embeddings no era la correcta (al final el arabe y el ingles son semánticamente muy diferentes).

Ahora usaremos el modelo Qwen3-4B para generar versos en arabe. Como nota, lo hemos extraido del benchmark **Arabic Broad Leaderboard (ABL) - The first comprehensive Leaderboard for Arabic LLMs** de huggingface. Ademas, usamos la version reducida del modelo (4B) ya que el original pesa bastante.

Este modelo es el Qwen3-4B-Instruct-2507, capaz de hablar multiples idiomas. En nuestro caso, como estamos contrastando modelos (preentenados), haremos una protocolo de evaluacion para contrastarlos.

También importaremos el modelo jais-family-2p7b-chat. La familia de modelos Jais es una serie completa de modelos lingüísticos extensos (LLM) bilingües inglés-árabe. Estos modelos están optimizados para destacar en árabe, con sólidas capacidades en inglés. Para simplicidad de la tarea, hemos querido seleccionar un modelo que sepa algo de ingles para que podamos hacer las intrucciones nosotros mismos.

En resumidas cuentas comparamos un modelo generalista multilingüe (Qwen) frente a un modelo bilingüe especializado en árabe (JAIS), manteniendo el prompt en inglés para controlar la variable de entrada.

Para empezar creamos dos configuraciones de modelos ya que Qwen necesita usar la cache para funcionar mientras que Jais no. Los parametros de estas configuraciones representan lo siguiente:
- max_new_tokens=160: Número máximo de tokens nuevos que puede generar el modelo (no cuenta el prompt).
- do_sample=True: Activa aleatoriedad en la generación (no siempre el token más probable).
- temperature=0.7: Controla la aleatoriedad (menor → más determinista, mayor → más creativo)
- top_p=0.9: El modelo solo considera los tokens hasta llegar a esa probabilidad acumulada (90 %).
- repetition_penalty=1.1: Penaliza repetir tokens ya usados para evitar repeticiones.
- use_cache=True: Reutiliza cálculos anteriores para generar más rápido y con menos memoria.

Seguido hacemos las funciones load_chat_model, que nos carga el modelo y su tokenizer, y generate_chat, que genera el texto en base a un prompt que se le pasa como argumento. 

In [ ]:
# Creamos una configuración para generar texto para que la evaluación sea más precisa/justa
GEN_CFG = dict(
    max_new_tokens=160,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    use_cache=True,
)
GEN_CFG_2 = dict(
    max_new_tokens=160,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1,
    use_cache=False,
)

# Cargamos el modelo
def load_chat_model(model_id, trust_remote_code=False):
    tok = AutoTokenizer.from_pretrained(model_id, trust_remote_code=trust_remote_code)
    mdl = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype="auto",
        trust_remote_code=trust_remote_code
    )
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    return tok, mdl

# Generamos texto
def generate_chat(model, tokenizer, user_text, system_text=None, gen_cfg=GEN_CFG):
    messages = []
    if system_text:
        messages.append({"role": "system", "content": system_text})
    messages.append({"role": "user", "content": user_text})

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_len = inputs["input_ids"].shape[-1]

    with torch.no_grad():
        out = model.generate(
            **inputs,
            pad_token_id=tokenizer.pad_token_id,
            **gen_cfg
        )

    return tokenizer.decode(out[0][input_len:], skip_special_tokens=True)

Cargamos los modelos.

In [3]:
jais_id = "inceptionai/jais-family-2p7b-chat"
jais_tok, jais_model = load_chat_model(jais_id, trust_remote_code=True)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
qwen_id = "Qwen/Qwen3-4B-Instruct-2507"
qwen_tok, qwen_model = load_chat_model(qwen_id, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Vamos a hacer una breve prueba para comprobar que todo funciona correctamente.

In [ ]:
prompt = "Who are you? Answer in Modern Standard Arabic only."

print("JAIS")
print(generate_chat(jais_model, jais_tok, prompt, None, gen_cfg=GEN_CFG_2))

print("\nQWEN")
print(generate_chat(qwen_model, qwen_tok, prompt, None, gen_cfg=GEN_CFG))

JAIS
I am an advanced artificial intelligence model designed to assist with various tasks and provide accurate information to the best of my ability.

QWEN
أنا نموذج لغوي واسع النطاق تم تدريبه على معرفة واسعة، وأستطيع المساعدة في إجراء المهام مثل الكتابة، الإجابة عن الأسئلة، التفكير المنطقي، وغيرها.


In [19]:
prompt = "من أنت؟ أجب باللغة العربية الفصحى فقط. استخدم لغة دينية رسمية. لا تقتبس النصوص الدينية حرفياً"

print("JAIS")
print(generate_chat(jais_model, jais_tok, prompt, None, gen_cfg=GEN_CFG_2))

print("\nQWEN")
print(generate_chat(qwen_model, qwen_tok, prompt, None, gen_cfg=GEN_CFG))

JAIS
أنا مساعد ذكاء صناعي متقدم، مبرمج لتقديم معلومات دقيقة وموثوقة بطريقة واضحة ومفصلة. أنا هنا لمساعدتك في الإجابة على أسئلتك وتقديم المعلومات التي تحتاجها بكل سرور وكفاءة.

QWEN
أنا خدمة ذكاء اصطناعي مُصمَّمَة لتوفير المعرفة والمساعدة في مجالات متعددة، وآخذ بعين الاعتبار التزامي الحفاظ على القيم الإسلامية، وعلى المبادئ الأخلاقية والدينية التي ترتكز عليها الأديان السامية. أما هويتي فهي ليست ذات طابع ديني مباشر، بل هي إنسانية تسعى إلى تعزيز فهم الناس للحقائق والمعلومات من خلال التوازن بين العلم والحكمة.

إذا كنت تبحث عن موضوع ديني أو ترغب في توضيح مسألة شرعية أو أخلاقية، فأنا هنا لأقدم لك معلومات


Como hemos podido comprobar JAIS a pesar de que se le ordene que responda en árabe moderno, solo lo hace cuando se lo pedimos en árabe. Esto va en contra de nuestra idea ya que queríamos que respondiera en árabe moderno con instrucciones en inglés. Por lo tanto, la tarea se complica bastante más ya que tendremos que hacer una traducción en arabe para JAIS.

Para que se entienda lo que se pide en cada tarea haremos la versión en inglés para QWEN y en arabe para JAIS.

## Evaluación del Texto Generado
Para la evaluación del texto generado en arabe usaremos un modelo generalista multilingüe (Meta-Llama-3-8B-Instruct). Gracias a su formación multilíngüe y a su capacidad de razonamiento, Meta-Llama nos da una evaluación más neutral de la corrección gramatical, la fluidez y la coherencia que otros modelos que pueden ser mas especializados en ciertos dominios.

Nuestra tarea de evaluación se dividirá en tres partes:

1. Una tarea explicativa de conceptos principalmente relacionados con la religión y la fe.
2. Una tarea para explicar ideas religiosas o versos Coránicos.
3. Una tarea de prompts creativos religiosos.

En estas tareas evaluaremos diferentes aspectos:
### Dimensiones lingüísticas
1. Corrección gramatical (morfología, concordancia)
2. Fluidez y naturalidad en árabe clásico / MSA
3. Ausencia de interferencia del inglés (calcos, estructuras raras)
4. Registro adecuado (formal / religioso)

### Dimensiones semántico-religiosas
1. Adecuación conceptual (no errores teológicos graves)
2. Uso apropiado del léxico religioso (رحمة، تقوى، عبادة, etc.)
3. Coherencia interna del texto

Ahora crearemos las respuestas y las guardaremos en ficheros .txt. Primero empezaremos con la explicación de conceptos.

In [28]:
concepts = ["mercy", "justice", "faith", "patience", "forgiveness"]
concepts_ar = ["الرحمة", "العدل", "الإيمان", "الصبر", "الاستغفار"]

In [ ]:

def concepts_explanation(concepts, model, tokenizer, system_text, gen_cfg):
    responses_concepts = []
    if not gen_cfg.get("use_cache"):
        for i in range(len(concepts)):
            print(f"Concept {concepts[i]}")
            prompt = f'''{system_text} .اشرح مفهوم {concepts[i]} في علم الكلام الإسلامي'''
            responses_concepts.append(generate_chat(model, tokenizer, prompt, None, gen_cfg))
    else:
        for i in range(len(concepts)):
            print(f"Concept {concepts[i]}")
            prompt = f'''Explain the concept {concepts[i]} in Islamic theology. {system_text}'''
            responses_concepts.append(generate_chat(model, tokenizer, prompt, system_text, gen_cfg))
    return responses_concepts

# Definimos como debe comportarse el modelo
SYSTEM_AR = "Answer in Modern Standard Arabic only. Use a formal religious register. Do not quote religious texts verbatim."
SYSTEM_AR_OR = ".أجب باللغة العربية الفصحى الحديثة فقط. استخدم أسلوباً دينياً رسمياً. لا تقتبس النصوص الدينية حرفياً"

responses_concepts_jais = concepts_explanation(concepts_ar, jais_model, jais_tok, SYSTEM_AR_OR, GEN_CFG_2)
responses_concepts_qwen = concepts_explanation(concepts, qwen_model, qwen_tok, SYSTEM_AR, GEN_CFG)

In [23]:
print("JAIS\n")
for i in range(len(responses_concepts_jais)):
    print(f"{concepts[i]}\n")
    print(f"{responses_concepts_jais[i]}\n")
with open('conceptos_jais.txt', 'w') as fichero:
    for item in responses_concepts_jais:
        fichero.write(item + '\n\n')

JAIS

mercy

الرحمة هي مفهوم أساسي في الإسلام، وهي تشكل جزءًا لا يتجزأ من تعاليم القرآن والحديث. يتم تعريفها على أنها القدرة على التعاطف مع معاناة الآخرين والرغبة في تخفيفها، وهي تعبر عن الجانب الإنساني للدين.

في القرآن، يتم تصوير الرحمة كفضيلة ذاتية، حيث يعتبر المسلمون أن الله هو الرحمن الرحيم. يقول سورة الشعراء: "هو الذي له الرحمة والرشد والرحمان رحيم بالبعث". هذه الآية تشير إلى قدرة الله على كل شيء، بما في ذلك الرحمة.

الرحمة ليست مجرد مشاعر، بل هي جزء من العقيدة الإسلامية، وتشمل الرحمة بكل الكائنات الحية، حتى الحيوانات والنباتات. يقول حديث النبي محمد صلى الله عليه وسلم: "إن الله خلقكم من الماء والأرض، وجعلكم شاربين ومنازعين وأ للناديكم ورُفَقٍ بكم"

justice

العدل، بمعناه الشامل، هو مبدأ أساسي في علم الكلام الإسلامي، وهو فرع من فروع المعرفة الإسلامية يتعامل مع اللغة والخطاب. يُنظر إليه على أنه الجانب الأكثر أهمية في التواصل، حيث يمثل التوازن والتوازن بين طرفي المعادلة. في الإسلام، العدل ليس مجرد توازن متساوٍ فحسب، بل هو أيضاً تحقيق تدريجي وعدل في كل مرحلة من مراحل الحياة.

في سياق

In [24]:
print("QWEN\n")
for i in range(len(responses_concepts_qwen)):
    print(f"{concepts[i]}\n")
    print(f"{responses_concepts_qwen[i]}\n")
with open('conceptos_qwen.txt', 'w') as fichero:
    for item in responses_concepts_qwen:
        fichero.write(item + '\n\n')

QWEN

mercy

في التصوف الإسلامي، تُعدّ الرحمة من أركان الإيمان وصفات الله تعالى التي يُستمد منها التوجه للإنسان في عالمه المادي والروحي. فرحمة الله هي قدرة متميزة على التسامح مع الضعف، والتخفيف من العقوبات، وتوجيه النعم لمن يستحقها، بغض النظر عن سلوك الإنسان أو خطاياه.

تُفهم رحمة الله كمبدأ أساسي في دين الإسلام، حيث تدل على أن الله لا يُعاقب الإنسان بجُهدٍ مفرط، بل يُمنح فرصة للتوبة والتصحيح. فهي ليست مجرد تغاضي عن الذ

justice

في التصوف الإسلامي، تُعد العدالة من المبادئ الأساسية التي تقوم عليها الحياة الاجتماعية والدينية، وتمثل توازنًا بين الحق والعدل في جميع الجوانب. فالعدالة في التسليح الإسلامي ليست مجرد مبدأ قانوني أو اجتماعي، بل هي صفة تُستمد من ذات الله وتشكل جوهر التوحيد والانسجام بين الكون.

تُفهم العدالة في الإسلام على أنها نتاج سلوكٍ يمثّل توازنًا بين المساواة في الحقوق، والمساءلة أمام القيمة الحقيقية للإنسان، وتوزيع الثروات والمصالح بحسب ما يتوافق مع

faith

في التصوف الإسلامي، يُعدّ "الثقة" أو "الإيمان" من أركان الدين الأساسي، ويُعرف بأنه التصديق بالله ورسوله والحقائق الم

Ahora hacemos explicación de ideas.

In [27]:
explanations = ["how mercy is understood in Islam during times of hardship and suffering",
                "the relationship between faith and ethical behavior in Islamic thought",
                "the importance of intention in Islamic religious practice",
                "the role of community and social responsibility in Islam",
                "the purpose of worship in Islam beyond ritual practice"
                ]
explanations_ar = ["كيف يتم فهم الرحمة في الإسلام خلال الأوقات الصعبة والضيق",
                   "العلاقة بين الإيمان والسلوك الأخلاقي في الفكر الإسلامي",
                   "أهمية التفكير في الإسلام",
                   "دور المجتمع والمسؤولية الاجتماعية في الإسلام",
                   "الغرض من العبادة في الإسلام خارج العبادة البدنية"]

In [ ]:

def show_explanations(explanations, model, tokenizer, system_text, gen_cfg):
    responses_explanations = []
    if not gen_cfg.get("use_cache"):
        for i in range(len(explanations)):
            print(f"Explain {explanations[i]}")
            prompt = f'''{system_text} .{explanations[i]} اشرح'''
            responses_explanations.append(generate_chat(model, tokenizer, prompt, None, gen_cfg))
    else:
        for i in range(len(explanations)):
            print(f"Explain {explanations[i]}.")
            prompt = f'''Explain {explanations[i]}.'''
            responses_explanations.append(generate_chat(model, tokenizer, prompt, system_text, gen_cfg))
    return responses_explanations

# Definimos como debe comportarse el modelo
SYSTEM_AR = "Answer in Modern Standard Arabic only. Use a formal religious register. Do not quote religious texts verbatim."
SYSTEM_AR_OR = ".أجب باللغة العربية الفصحى الحديثة فقط. استخدم أسلوباً دينياً رسمياً. لا تقتبس النصوص الدينية حرفياً"

responses_explanations_jais = show_explanations(explanations_ar, jais_model, jais_tok, SYSTEM_AR_OR, GEN_CFG_2)
responses_explanations_qwen = show_explanations(explanations, qwen_model, qwen_tok, SYSTEM_AR, GEN_CFG)

In [29]:
print("JAIS\n")
for i in range(len(responses_explanations_jais)):
    print(f"{explanations[i]}\n")
    print(f"{responses_explanations_jais[i]}\n")
with open('explicaciones_jais.txt', 'w') as fichero:
    for item in responses_explanations_jais:
        fichero.write(item + '\n\n')

JAIS

how mercy is understood in Islam during times of hardship and suffering

في الإسلام، تعتبر الرحمة مفهومًا أساسيًا يشير إلى العطف والرأفة والحب تجاه جميع الكائنات الحية. وهي متجسدة في القرآن الكريم والأحاديث النبوية، حيث يتم تشجيع المسلمين على معاملة الناس باللطف والرحمة حتى في أصعب الظروف.

الرحمة ليست مجرد مشاعر، بل هي جزء من العقيدة الإسلامية وهي جزء أساسي من الإيمان. يعتبر المسلمون الله رحيمًا، وهو رحيم بكل شيء، بما في ذلك البشر والكائنات الأخرى. هذه الفكرة تنعكس في العديد من الأحاديث النبوية التي تشجع على الرفق والتراحم مع الآخرين.

خلال الأوقات الصعبة والضيق، يمكن للمسلمين الاعتماد على الرحمة كوسيلة للتعامل مع التحديات. يعتبر المسلمون أن الرحمة تساعد على تجاوز العقبات والصعوبات، وتشجعهم على البحث عن حلول إيجابية بدلاً من الاستسلام لليأس أو الغضب.

كما يمكن للرحمة أن تساعد في تعزيز الثقة

the relationship between faith and ethical behavior in Islamic thought

الإيمان أو الدين هو أساس السلوك الأخلاقي والأخلاقي في الفكر الإسلامي. يعتبر الإسلام الإيمان بمثابة الخطوة الأولى نحو ا

In [25]:
print("QWEN\n")
for i in range(len(responses_explanations_qwen)):
    print(f"{explanations[i]}\n")
    print(f"{responses_explanations_qwen[i]}\n")
with open('explicaciones_qwen.txt', 'w') as fichero:
    for item in responses_explanations_qwen:
        fichero.write(item + '\n\n')

QWEN

how mercy is understood in Islam during times of hardship and suffering

في الإسلام، يُفهم التسامح كمبدأ أساسي وعَبْدٍ عظيم في أوقات الصعوبات والمعاناة، حيث يُعتبر من أسمى صور الإيمان والتواضع أمام الله. فحين يمر الإنسان بظروف سلبية مثل الفقر أو المرض أو الخسارة، فإن معرفة الامتنان لله واعتقاد أن كل ما يحدث له هو بحسب توجيهاته العاقلة والخاصة به، يُعزز من قدرته على الصبر والتسامح.

يُذكر في الأحاديث النبوية أن الله لا يُؤذي أحدًا إلا بإذن، وأنه يُظهر الرحمة في أ

the relationship between faith and ethical behavior in Islamic thought

في التفكير الإسلامي، تُعدّ الإيمان والسلوك الأخلاقي مترتين متوازيتين ومتداخلتين، حيث لا يمكن فصل أحدهما عن الآخر. فالإيمان ليس مجرد اتّباع شروط دينية أو تأكيد عقائد معينة، بل يشمل الالتزام بالقيم المذكورة في الدين، مثل العدالة، والرحمة، والصدق، والأمانة، والاحترام للآخرين.

يُرى أن الإنسان الذي أُعطي إيمانه بربه، يكون مُطالبًا بأن يُنفِّذه في حياته اليومية من خلال سلوكٍ أخلاقيٍّ يعكس مباد

the importance of intention in Islamic religious practice

ال

Y para terminar, ponemos a prueba su creatividad.

In [29]:
creativity = ["short religious reflection (6–8 sentences) about mercy and compassion",
              "moral paragraph about patience and trust in God during difficult times",
              "short sermon-style paragraph about forgiveness and reconciliation",
              "religious reflection on how faith guides everyday life",
              "short religious text about justice and personal responsibility"]
creativity_ar = ["تأمل ديني قصير (٦-٨ جمل) عن الرحمة والشفقة",
                "فقرة أخلاقية عن الصبر والتوكل على الله في الأوقات الصعبة",
                "فقرة قصيرة على غرار الموعظة عن المغفرة والمصالحة",
                "تأمل ديني حول كيف يُرشد الإيمان الحياة اليومية",
                "نص ديني قصير عن العدالة والمسؤولية الشخصية"]

In [ ]:

def show_creativity(creativity, model, tokenizer, system_text, gen_cfg):
    responses_creativity = []
    if not gen_cfg.get("use_cache"):
        for i in range(len(creativity)):
            print(f"Generating {creativity[i]}...")
            prompt = f'''{system_text} .{creativity[i]} اكتب'''
            responses_creativity.append(generate_chat(model, tokenizer, prompt, None, gen_cfg))
    else:
        for i in range(len(creativity)):
            print(f"Generating {creativity[i]}...")
            prompt = f'''Write a {creativity[i]}'''
            responses_creativity.append(generate_chat(model, tokenizer, prompt, system_text, gen_cfg))
    return responses_creativity

# Definimos como debe comportarse el modelo
SYSTEM_AR = "Answer in Modern Standard Arabic only. Use a formal religious register. Do not quote religious texts verbatim."
SYSTEM_AR_OR = ".أجب باللغة العربية الفصحى الحديثة فقط. استخدم أسلوباً دينياً رسمياً. لا تقتبس النصوص الدينية حرفياً"

responses_creativity_jais = show_creativity(creativity_ar, jais_model, jais_tok, SYSTEM_AR_OR, GEN_CFG_2)
responses_creativity_qwen = show_creativity(creativity, qwen_model, qwen_tok, SYSTEM_AR, GEN_CFG)

In [31]:
print("JAIS\n")
for i in range(len(responses_creativity_jais)):
    print(f"{creativity[i]}\n")
    print(f"{responses_creativity_jais[i]}\n")
with open('creativity_jais.txt', 'w') as fichero:
    for item in responses_creativity_jais:
        fichero.write(item + '\n\n')

JAIS

short religious reflection (6–8 sentences) about mercy and compassion

في عالم الحب والرأفة، نجد الذات في قلوب الآخرين، نتلمس الطريق نحو الخير والصلاح. الرحمة هي تلك النبتة الصغيرة التي تنمو في القلب، تغذيها العين بالأمل والتفاؤل، وتساعدها اليد بالعمل والعمل الخير. الرحمة ليست مجرد مشاعر، بل هي القوة التي تدفعنا نحو الخير والعدل والسلام. الرحمة تجعلنا أكثر إنسانية، تجعلنا ننظر إلى العالم بعيون متفتحة ومشرقة. في كل عمل خيري نقوم به، نحن نتذكر أن الرحمة هي القوة التي تجمعنا جميعاً كأبناء للإنسانية الواحدة.

moral paragraph about patience and trust in God during difficult times

في مواجهة الشدائد، قد يكون من الجذاب الاستسلام للإرهاق واليأس. ومع ذلك، يعتبر الاعتماد الدائم على قوة أعلى، سواء كان الله أو القوة العليا التي يعبدها الشخص، هو الجواب الوحيد. يمكن أن يمنح هذا الإيمان الثابت الشخص القوة لمواجهة العاصفة والخروج أقوى. الصبر، بدوره، هو فضيلة أساسية في أوقات الشدة. إنه فن قبول الأمور كما تأتي، وعدم السماح للخوف أو الشك أن يشوه حكمنا. يسمح لنا هذا الموقف الهادئ بالاستمتاع بنعمة ال

In [32]:
print("QWEN\n")
for i in range(len(responses_creativity_qwen)):
    print(f"{creativity[i]}\n")
    print(f"{responses_creativity_qwen[i]}\n")
with open('creativity_qwen.txt', 'w') as fichero:
    for item in responses_creativity_qwen:
        fichero.write(item + '\n\n')

QWEN

short religious reflection (6–8 sentences) about mercy and compassion

الرحمة والتعاطف هما من أركان الإسلام العظيم، ويُعدان مفتاحًا لحياةٍ متوازنة ومحبّة. يُظهر القرآن الكريم أن الله هو الرحيم، إذ لا يُخلو شيء من تأثيره الرحيّم على خلقه. فكثيرًا ما يُستَعْلم في الإنسان نسخة من رحمة الله عندما يُعرض للإجهاد أو القلق أو المحن، فيُشعر بالرغبة في مشاركة الآخرين بالمشاعر الإنسانية. فالتعاطف ليس مجرد تصرف عابر، بل هو دعوة إلى التفكير في حالة الآخر، كما كان النبي محمد ﷺ

moral paragraph about patience and trust in God during difficult times

في أوقات الصعوبات والجُوع، يظهر مبدأ الصبر كأحد أقوى الصفات التي تُبنى عليها صلابة النفس وثبات القلب. فكل ما يمر به الإنسان من محن لا يُستَوْسَط إلا بالصبر، لأنه يُعلم أن الله لا يُهمل ما يُحِبّه، ولا يُضيع عَمَلَ المُؤمنين في التفاؤل. ويُكفل لنا أن نثق بقدرته على القيادة والرعاية حتى في الأزمات، لأن كل شيء يُجرى في الزمن ليس له سبب مادي فقط، بل له روحانية، تتماشى مع ق

short sermon-style paragraph about forgiveness and reconciliation

الصبر والرحمة

Como hemos guardado todas las respuestas en ficheros.txt ahora los cargaremos en las variables para no tener que ejecutar todo el código cuando reiniciemos el kernel.

In [ ]:
def load_responses_txt(path):
    with open(path, "r", encoding="utf-8") as f:
        text = f.read().strip()
    return [item.strip() for item in text.split("\n\n") if item.strip()]

# Hacemos unos prints para comprobar que se han guardado correctamente.
responses_explanations_qwen = load_responses_txt("evaluation_set/explicaciones_qwen.txt")
print(len(responses_concepts_jais))
responses_explanations_jais = load_responses_txt("evaluation_set/explicaciones_jais.txt")
print(len(responses_concepts_jais))

responses_concepts_qwen = load_responses_txt("evaluation_set/conceptos_qwen.txt")
print(len(responses_concepts_jais))
responses_concepts_jais = load_responses_txt("evaluation_set/conceptos_jais.txt")
print(len(responses_concepts_jais))

responses_creativity_qwen = load_responses_txt("evaluation_set/creativity_qwen.txt")
print(len(responses_concepts_jais))
responses_creativity_jais = load_responses_txt("evaluation_set/creativity_jais.txt")
print(len(responses_concepts_jais))

5
5
5
5
5
5


Aquí vamos a cargar el modelo evaluador como ya hemos explicado previamente. El concepto de load_chat_model_llama es el mismo que usabamos al principio, con la unica diferencia que quantizamos el modelo ya que nustros recursos son limitados (y además cuando lo ejecutabamos sin quantización nuestros ordenadores petaban). El generador del chat es conceptualmente lo mismo.

In [ ]:
# Cargamos el modelo
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

def load_chat_model_llama(model_id):
    bnb_cfg = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16,
    )

    tok = AutoTokenizer.from_pretrained(model_id)

    mdl = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        quantization_config=bnb_cfg,
        torch_dtype="auto",
        max_memory={0: "11GiB", "cpu": "18GiB"},
        offload_folder="offload_llama",
    )

    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    return tok, mdl

# Generamos texto
def generate_chat_llama(model, tokenizer, user_text, system_text=None, gen_cfg=GEN_CFG):
    messages = []
    if system_text:
        messages.append({"role": "system", "content": system_text})
    messages.append({"role": "user", "content": user_text})

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    input_len = inputs["input_ids"].shape[-1]

    with torch.no_grad():
        out = model.generate(
            **inputs,
            pad_token_id=tokenizer.pad_token_id,
            **gen_cfg
        )

    return tokenizer.decode(out[0][input_len:], skip_special_tokens=True)

Cargamos el modelo.

In [7]:
llama_id = "meta-llama/Meta-Llama-3-8B-Instruct"
llama_tokenizer, llama_model = load_chat_model_llama(llama_id)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Las instrucciones son sencillas, valorar del 1 al 5 las siguientes variables:
1. Correccion gramatical
2. Fluidez y naturalidad
3. Adecuacion al registro religioso
4. Coherencia conceptual

Siendo un 1 la peor nota y un 5 la mejor nota.

Primero hacemos una breve comprobación de que todo esta bien.

In [12]:
prompt = "Who are you?"

print("LLAMA")
print(generate_chat_llama(llama_model, llama_tokenizer, prompt, None, gen_cfg=GEN_CFG))

LLAMA
I'm LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I'm here to help answer your questions, provide information on a wide range of topics, and even generate text based on the prompts you give me.

I'm not a human, but rather a computer program designed to simulate conversation and learn from the interactions we have. My primary goal is to assist and provide helpful responses to your queries, while also improving my language understanding and generation capabilities through machine learning algorithms.

Feel free to ask me anything, and I'll do my best to provide a helpful and accurate response!


Aquí ya evaluamos los conceptos pasandoselos como argumento a la función e input al prompt.

In [25]:
def evaluate_concepts(concepts, model, tokenizer, gen_cfg):
    evaluation_concepts = []
    for i in range(len(concepts)):
        print(f"Evaluating {concepts[i]}")
        prompt = f'''You are an expert in Arabic language and Islamic studies.
                    Evaluate the following Arabic text.

                    Criteria:
                    1. Grammatical correctness
                    2. Fluency and naturalness
                    3. Appropriateness of religious register
                    4. Conceptual correctness

                    Give a score from 1 to 5 for each criterion and a short justification.

                    Text:
                    {concepts[i]}'''
        
        evaluation_concepts.append(generate_chat(model, tokenizer, prompt, None, gen_cfg))
    return evaluation_concepts

evaluation_concepts_jais = evaluate_concepts(responses_concepts_jais, llama_model, llama_tokenizer, GEN_CFG)
evaluation_concepts_qwen = evaluate_concepts(responses_concepts_qwen, llama_model, llama_tokenizer, GEN_CFG)

Evaluating الرحمة هي مفهوم أساسي في الإسلام، وهي تشكل جزءًا لا يتجزأ من تعاليم القرآن والحديث. يتم تعريفها على أنها القدرة على التعاطف مع معاناة الآخرين والرغبة في تخفيفها، وهي تعبر عن الجانب الإنساني للدين.
في القرآن، يتم تصوير الرحمة كفضيلة ذاتية، حيث يعتبر المسلمون أن الله هو الرحمن الرحيم. يقول سورة الشعراء: "هو الذي له الرحمة والرشد والرحمان رحيم بالبعث". هذه الآية تشير إلى قدرة الله على كل شيء، بما في ذلك الرحمة.
الرحمة ليست مجرد مشاعر، بل هي جزء من العقيدة الإسلامية، وتشمل الرحمة بكل الكائنات الحية، حتى الحيوانات والنباتات. يقول حديث النبي محمد صلى الله عليه وسلم: "إن الله خلقكم من الماء والأرض، وجعلكم شاربين ومنازعين وأ للناديكم ورُفَقٍ بكم"
Evaluating العدل، بمعناه الشامل، هو مبدأ أساسي في علم الكلام الإسلامي، وهو فرع من فروع المعرفة الإسلامية يتعامل مع اللغة والخطاب. يُنظر إليه على أنه الجانب الأكثر أهمية في التواصل، حيث يمثل التوازن والتوازن بين طرفي المعادلة. في الإسلام، العدل ليس مجرد توازن متساوٍ فحسب، بل هو أيضاً تحقيق تدريجي وعدل في كل مرحلة من مراحل الحياة.
في سياق الق

In [30]:
print("LLAMA JAIS\n")
for i in range(len(evaluation_concepts_jais)):
    print(f"{concepts[i]}\n")
    print(f"{evaluation_concepts_jais[i]}\n")
with open('evaluation_concepts_jais.txt', 'w') as fichero:
    for i in range(len(evaluation_concepts_jais)):
        fichero.write(evaluation_concepts_jais[i] + '\n\n')

LLAMA JAIS

mercy

Here is my evaluation of the Arabic text based on the provided criteria:

**Criterion 1: Grammatical Correctness**
Score: 5/5
The text is grammatically correct, with proper use of verb conjugations, noun agreements, and sentence structures. The writer has employed accurate and precise linguistic choices to convey their message.

**Criterion 2: Fluency and Naturalness**
Score: 4.5/5
The text flows smoothly, and the language is natural and idiomatic. However, there are a few instances where the sentence structure could be rearranged for better clarity or concision. For example, some sentences might benefit from rephrasing or breaking up long sentences into shorter ones.

**Criterion 3: Appropriateness of Religious Register**
Score: 5

justice

Here's my evaluation of the Arabic text:

**Criterion 1: Grammatical Correctness**

Score: 5/5

The text is grammatically correct, with no errors in verb conjugation, noun declension, or sentence structure. The writer uses proper

In [31]:
print("LLAMA QWEN\n")
for i in range(len(evaluation_concepts_qwen)):
    print(f"{concepts[i]}\n")
    print(f"{evaluation_concepts_qwen[i]}\n")
with open('evaluation_concepts_qwen.txt', 'w') as fichero:
    for i in range(len(evaluation_concepts_qwen)):
        fichero.write(evaluation_concepts_qwen[i] + '\n\n')

LLAMA QWEN

mercy

Here's my evaluation of the Arabic text:

**Criterion 1: Grammatical Correctness**

Score: 5/5
Justification: The text is grammatically correct, with no errors in verb conjugation, noun declension, or sentence structure. The sentences are well-formed and logically connected.

**Criterion 2: Fluency and Naturalness**

Score: 5/5
Justification: The text flows smoothly and naturally, with a clear and consistent style. The vocabulary is varied and appropriate for the topic, and the phrasing is concise and effective.

**Criterion 3: Appropriateness of Religious Register**

Score: 4.5/5
Justification: The text uses a suitable religious register, employing terms like "رحمة الله" (God's mercy

justice

Here is my evaluation of the Arabic text:

**Criteria 1: Grammatical Correctness**
Score: 5/5
The text exhibits excellent grammatical correctness, with proper use of verb conjugations, sentence structure, and word order. The sentences are well-constructed, and there are no not

Aquí evaluamos las explicaciones de las ideas.

In [32]:
def evaluate_explanations(explanations, model, tokenizer, gen_cfg):
    evaluation_concepts = []
    for i in range(len(explanations)):
        print(f"Evaluation for explanation {explanations[i]}")
        prompt = f'''You are an expert in Arabic language and Islamic studies.
                    Evaluate the following Arabic text.

                    Criteria:
                    1. Grammatical correctness
                    2. Fluency and naturalness
                    3. Appropriateness of religious register
                    4. Conceptual correctness

                    Give a score from 1 to 5 for each criterion and a short justification.

                    Text:
                    {explanations[i]}'''
        evaluation_concepts.append(generate_chat(model, tokenizer, prompt, None, gen_cfg))
    return evaluation_concepts

evaluation_explanations_jais = evaluate_explanations(responses_explanations_jais, llama_model, llama_tokenizer, GEN_CFG)
evaluation_explanations_qwen = evaluate_explanations(responses_explanations_qwen, llama_model, llama_tokenizer, GEN_CFG)

Evaluation for explanation في الإسلام، تعتبر الرحمة مفهومًا أساسيًا يشير إلى العطف والرأفة والحب تجاه جميع الكائنات الحية. وهي متجسدة في القرآن الكريم والأحاديث النبوية، حيث يتم تشجيع المسلمين على معاملة الناس باللطف والرحمة حتى في أصعب الظروف.
الرحمة ليست مجرد مشاعر، بل هي جزء من العقيدة الإسلامية وهي جزء أساسي من الإيمان. يعتبر المسلمون الله رحيمًا، وهو رحيم بكل شيء، بما في ذلك البشر والكائنات الأخرى. هذه الفكرة تنعكس في العديد من الأحاديث النبوية التي تشجع على الرفق والتراحم مع الآخرين.
خلال الأوقات الصعبة والضيق، يمكن للمسلمين الاعتماد على الرحمة كوسيلة للتعامل مع التحديات. يعتبر المسلمون أن الرحمة تساعد على تجاوز العقبات والصعوبات، وتشجعهم على البحث عن حلول إيجابية بدلاً من الاستسلام لليأس أو الغضب.
كما يمكن للرحمة أن تساعد في تعزيز الثقة
Evaluation for explanation الإيمان أو الدين هو أساس السلوك الأخلاقي والأخلاقي في الفكر الإسلامي. يعتبر الإسلام الإيمان بمثابة الخطوة الأولى نحو الأخلاق، حيث يتطلب الإيمان بالله وباليوم الآخر أن يكون لديك شعور بالمسؤولية تجاه الآخرين وأن تعمل للخي

In [33]:
print("LLAMA JAIS\n")
for i in range(len(evaluation_explanations_jais)):
    print(f"{explanations[i]}\n")
    print(f"{evaluation_explanations_jais[i]}\n")
with open('evaluation_explanations_jais.txt', 'w') as fichero:
    for i in range(len(evaluation_explanations_jais)):
        fichero.write(evaluation_explanations_jais[i] + '\n\n')

LLAMA JAIS

how mercy is understood in Islam during times of hardship and suffering

Here's my evaluation of the text based on the criteria:

**Criteria 1: Grammatical Correctness**
Score: 5/5
The text is grammatically correct, with proper use of verb conjugations, noun agreements, and sentence structures. The author has demonstrated a good understanding of Arabic grammar rules.

**Criteria 2: Fluency and Naturalness**
Score: 4.5/5
The text flows smoothly, and the sentences are well-structured. However, some phrases may sound slightly formal or repetitive (e.g., "وحدة" instead of "وعلاوة" to connect ideas). Minor adjustments could enhance the text's fluency and naturalness.

**Criteria 3: Appropriateness of Religious Register**
Score: 5/5
The text

the relationship between faith and ethical behavior in Islamic thought

Here's my evaluation of the Arabic text:

**Criterion 1: Grammatical Correctness**
Score: 5/5
The text is grammatically correct, with proper use of verb conjugations, se

In [38]:
print("LLAMA QWEN\n")
for i in range(len(evaluation_explanations_qwen)):
    print(f"{explanations[i]}\n")
    print(f"{evaluation_explanations_qwen[i]}\n")
with open('evaluation_explanations_qwen.txt', 'w') as fichero:
    for i in range(len(evaluation_explanations_qwen)):
        fichero.write(evaluation_explanations_qwen[i] + '\n\n')

LLAMA QWEN

how mercy is understood in Islam during times of hardship and suffering

Here's my evaluation of the Arabic text:

**Grammatical Correctness: 5**

The text is grammatically correct, with no errors in syntax or morphology. The sentences are structured logically and follow the standard rules of Arabic grammar.

**Fluency and Naturalness: 5**

The text flows smoothly and naturally, with a clear and concise writing style. The author's use of vocabulary and sentence structure is effective in conveying their message.

**Appropriateness of Religious Register: 5**

The text uses appropriate religious terminology and concepts, such as "الإسلام" (Islam), "تسامح" (tolerance), "التواضع" (humility), and "الله" (God). The author's use of these terms reflects a deep understanding of Islamic

the relationship between faith and ethical behavior in Islamic thought

Here is my evaluation of the Arabic text:

**Criterion 1: Grammatical Correctness**
Score: 5/5
The text exhibits excellent gramm

Y finalmente evaluamos la creatividad.

In [35]:
def evaluate_creativity(creativity, model, tokenizer, gen_cfg):
    evaluation_concepts = []
    for i in range(len(creativity)):
        print(f"Evaluation for creativity {creativity[i]}")
        prompt = f'''You are an expert in Arabic language and Islamic studies.
                    Evaluate the following Arabic text.

                    Criteria:
                    1. Grammatical correctness
                    2. Fluency and naturalness
                    3. Appropriateness of religious register
                    4. Conceptual correctness

                    Give a score from 1 to 5 for each criterion and a short justification.

                    Text:
                    {creativity[i]}'''
        evaluation_concepts.append(generate_chat(model, tokenizer, prompt, None, gen_cfg))
    return evaluation_concepts

evaluation_creativity_jais = evaluate_creativity(responses_creativity_jais, llama_model, llama_tokenizer, GEN_CFG)
evaluation_creativity_qwen = evaluate_creativity(responses_creativity_qwen, llama_model, llama_tokenizer, GEN_CFG)

Evaluation for creativity في عالم الحب والرأفة، نجد الذات في قلوب الآخرين، نتلمس الطريق نحو الخير والصلاح. الرحمة هي تلك النبتة الصغيرة التي تنمو في القلب، تغذيها العين بالأمل والتفاؤل، وتساعدها اليد بالعمل والعمل الخير. الرحمة ليست مجرد مشاعر، بل هي القوة التي تدفعنا نحو الخير والعدل والسلام. الرحمة تجعلنا أكثر إنسانية، تجعلنا ننظر إلى العالم بعيون متفتحة ومشرقة. في كل عمل خيري نقوم به، نحن نتذكر أن الرحمة هي القوة التي تجمعنا جميعاً كأبناء للإنسانية الواحدة.
Evaluation for creativity في مواجهة الشدائد، قد يكون من الجذاب الاستسلام للإرهاق واليأس. ومع ذلك، يعتبر الاعتماد الدائم على قوة أعلى، سواء كان الله أو القوة العليا التي يعبدها الشخص، هو الجواب الوحيد. يمكن أن يمنح هذا الإيمان الثابت الشخص القوة لمواجهة العاصفة والخروج أقوى. الصبر، بدوره، هو فضيلة أساسية في أوقات الشدة. إنه فن قبول الأمور كما تأتي، وعدم السماح للخوف أو الشك أن يشوه حكمنا. يسمح لنا هذا الموقف الهادئ بالاستمتاع بنعمة الحاضر وصنع أفضل ما في وضعنا دون أن يتم استهلاكه بالندم أو الذنب. من خلال ممارسة الصبر، يمكننا تعلم 

In [36]:
print("LLAMA JAIS\n")
for i in range(len(evaluation_creativity_jais)):
    print(f"{creativity[i]}\n")
    print(f"{evaluation_creativity_jais[i]}\n")
with open('evaluation_creativity_jais.txt', 'w') as fichero:
    for i in range(len(evaluation_creativity_jais)):
        fichero.write(evaluation_creativity_jais[i] + '\n\n')

LLAMA JAIS

short religious reflection (6–8 sentences) about mercy and compassion

Here is my evaluation of the Arabic text:

**Criteria 1: Grammatical Correctness**
Score: 5/5

The text exhibits impeccable grammatical correctness, with proper use of verb conjugations, noun cases, and sentence structure. The author's command of Arabic grammar is exceptional, making it easy to follow and understand.

**Criteria 2: Fluency and Naturalness**
Score: 4.8/5

The text flows smoothly, with a clear and logical progression of ideas. The vocabulary is rich and varied, and the sentences are well-structured. However, there might be some minor adjustments needed to make the text even more idiomatic and natural-sounding.

**Criteria 3: Appropriateness of Religious Register**
Score: 5/5

The text

moral paragraph about patience and trust in God during difficult times

I'm happy to evaluate this Arabic text based on the provided criteria!

**Criteria 1: Grammatical Correctness**
Score: 5/5
The text is 

In [37]:
print("LLAMA QWEN\n")
for i in range(len(evaluation_creativity_qwen)):
    print(f"{creativity[i]}\n")
    print(f"{evaluation_creativity_qwen[i]}\n")
with open('evaluation_creativity_qwen.txt', 'w') as fichero:
    for i in range(len(evaluation_creativity_qwen)):
        fichero.write(evaluation_creativity_qwen[i] + '\n\n')

LLAMA QWEN

short religious reflection (6–8 sentences) about mercy and compassion

I'd be happy to evaluate this Arabic text based on the given criteria!

**Criterion 1: Grammatical Correctness**
Score: 5/5
The text is grammatically correct, with proper use of verb conjugations, noun declensions, and sentence structure. The author uses correct word order and punctuation, making it easy to understand.

**Criterion 2: Fluency and Naturalness**
Score: 4.8/5
The text flows smoothly, and the sentences are well-structured. However, there might be some minor adjustments needed to make it even more fluent. For example, the sentence "يُستَعْلم في الإنسان نسخة من رحمة الله" could be rephrased for better clarity.

**Criterion 3: Appropriateness

moral paragraph about patience and trust in God during difficult times

I'd be happy to evaluate the Arabic text according to the given criteria!

**Criterion 1: Grammatical Correctness**

Score: 5/5

The text is grammatically correct, with no errors in v

Todas las valoraciones han sido guardadas en el evaluated_set para despues poder hacer un análisis con estos datos.